## 6.2 순환 신경망(RNN) 이해하기

- [[링크](http://excelsior-cjh.tistory.com/183)] 참고
- **순환 신경망**(RNN, Recurrent Neural Network)는 시퀀스의 원소를 순회하면서 지금까지 처리한 정보를 **상태(state)**에 저장한다.
- RNN은 내부에 루프(loop)을 가진 신경망의 한 종류다.

$$
\begin{align*}
h_t &= \tanh \left(\mathbf{X}_{t} \cdot \mathbf{W}_{x} + h_{t-1} \cdot \mathbf{W}_{h} + \mathbf{b} \right) \\ \mathbf{Y}_{t} &= \mathbf{W}^{T}_{y} \cdot h_{t}
\end{align*}
$$

![](./images/rnn04.png)

### NumPy를 이용한 RNN 구현하기

1. 이 RNN은 형태가 `(timesteps, input_features)`인 2D인 텐서로 인코딩된 벡터의 시퀀스를 입력받는다.
2. 이 시퀀스는 타임스텝(`timesteps`)을 따라 반복된다.
3. 각 타임스텝 `t`에서 현재 상태(`state`)와 입력을 연결하여 출력을 계산한다.
4. 이 출력을 다음 스텝(`t+1`)의 상태로 설정한다.
    - 첫 번째 타임스텝에서는 출력이 정의되어있지 않으므로 **초기 상태**인 0벡터로 상태를 초기화한다.

In [3]:
import numpy as np

timesteps = 100  # 입력 시퀀스에 있는 타임스텝의 수
input_features = 32  # 입력 특성의 차원
output_features = 64  # 출력 특성의 차원

inputs = np.random.random((timesteps, input_features))  # 입력 데이터(X): 예제를 위해 생성한 난수

state_t = np.zeros((output_features,))  # 초기 상태(h0): 모두 0인 벡터

W_x = np.random.random((output_features, input_features))
W_h = np.random.random((output_features, output_features))
b = np.random.random((output_features,))

outputs = []
for input_t in inputs:  # input_t는 형태가 (input_featurs,)인 벡터다
    # 입력(x_t)과 현재 상태(이전 출력, h_{t-1})을 연결하여 현재 출력(h_t)를 얻는다.
    h_t = np.tanh(np.dot(W_x, input_t) + np.dot(W_h, state_t) + b)
    outputs.append(h_t)  # 이 출력을 리스트에 저장
    state_t = h_t  # 다음 타임스텝을 위해 네트워크의 상태를 업데이트

# 최종 출력은 형태가 (timestpes, output_features) = (100, 64)
final_output_sequence = np.stack(outputs, axis=0)

In [4]:
final_output_sequence.shape

(100, 64)

---

In [1]:
import keras

keras.__version__

Using TensorFlow backend.


'2.2.4'

### 6.2.1 케라스의 순환 층 - `SimpleRNN`

넘파이로 간단하게 구현한 과정이 실제 케라스의 [`SimpleRNN`](https://keras.io/layers/recurrent/#simplernn) 층에 해당합니다:

In [2]:
from keras.layers import SimpleRNN

케라스에 있는 모든 순환 층과 동일하게 SimpleRNN은 두 가지 모드로 실행할 수 있습니다. 

- 입력 시퀀스에 대한 마지막 출력만 반환할 수 있습니다(크기가 `(batch_size, output_features)` 인 2D 텐서). 
- 각 타임스텝의 출력을 모은 전체 시퀀스를 반환하거나(크기가 `(batch_size, timesteps, output_features)` 인 3D 텐서)
    - 이 모드는 객체를 생성할 때 `return_sequences` 매개변수로 선택할 수 있습니다. 
    - return_sequences: Boolean. Whether to return the last output
        in the output sequence, or the full sequence.

예제를 살펴보죠:

#### 마지막 타임스텝의 출력만 얻기

In [3]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))  # 32 * 32 * 2(W_x, W_h) + 32
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


#### 전체 상태 시퀀스를 얻기

In [4]:
from keras import backend as K

K.clear_session()

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


네트워크의 표현력을 증가시키기 위해 여러 개의 순환 층을 차례대로 쌓는 것이 유용할 때가 있습니다. 
- 이런 설정에서는 **중간 층들이 전체 출력 시퀀스를 반환하도록 설정**해야 합니다:

![](./images/deep-rnn.PNG)

In [5]:
K.clear_session()

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))  # 맨 위 층만 마지막 출력만 반환
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                2080      
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________


#### IMDB 영화 리뷰 분류 문제에 적용하기

이제 IMDB 영화 리뷰 분류 문제에 적용해 보죠. 먼저 데이터를 전처리합니다:

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # 특성으로 사용할 단어의 수
maxlen = 500  # 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용)
batch_size = 32

print